# pyspark1
Создаем простой DataFrame и пробуем на нем функции

Несколько полезных ссылок, прежде, чем начнем:  
- Jupyter Notebook:  
http://83.166.236.17:8000/user/v.alehin/notebooks/
- Yarn:  
http://91.219.226.252:8088/cluster
http://91.219.226.252:8088/cluster/scheduler
- App info:  
http://91.219.226.252:8088/cluster/app/{application_id}/
- Spark:       
http://91.219.226.252:8088/proxy/{application_id}/stages/
- Luigi visualizer:  
http://83.166.236.17:8082/static/visualiser/index.html
- Airflow:  
http://83.166.236.17:8080/admin/

In [ ]:
# pyspark1
# Создаем простой DataFrame и пробуем на нем функции

import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

os.environ["HADOOP_CONF_DIR"]="/etc/hadoop/conf"
# os.environ["SPARK_HOME"]="/usr/hdp/current/spark2-client"
# os.environ["JAVA_HOME"]="/usr/java/jdk1.8.0_191/jre"

spark = pyspark.sql.SparkSession.builder\
    .master("yarn")\
    .appName("v_alehin_pyspark1")\
    .config("spark.executor.instances", "1")\
    .config("spark.executor.memory", "1G")\
    .config("spark.executor.cores", "2")\
    .config("spark.dynamicAllocation.enabled", "false")\
    .config("spark.dynamicAllocation.executorIdleTimeout", "300s")\
    .config("spark.dynamicAllocation.maxExecutors", "1000")\
    .config("spark.driver.memory", "1G")\
    .config("spark.driver.maxResultSize", "1G")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "1024m")\
    .getOrCreate()

spark_context = spark.sparkContext

In [ ]:
print("""
Urls:
Yarn       http://91.219.226.252:8088/cluster/scheduler
Spark      http://91.219.226.252:8088/proxy/{app_id}/stages/
App info   http://91.219.226.252:8088/cluster/app/{app_id}/
""".format(app_id=spark_context.applicationId))

In [ ]:
data = [
    [1, "MaiL", 100.0],
    [2, "GooGle", 93.4],
    [3, "YaNDEX", 59.9],
    [4, "sPUtnik", 1.33],
]

In [ ]:
from pyspark.sql.types import *

dfSchema = StructType([
    StructField("id", IntegerType()),
    StructField("company_name", StringType()),
    StructField("score", DoubleType()),
])

In [ ]:
df = spark.createDataFrame(data, schema=dfSchema)

In [ ]:
df.show(5, False)

In [ ]:
df = spark.createDataFrame(data, schema=dfSchema)

In [ ]:
df.show(5, False)

In [ ]:
df.printSchema()

In [ ]:
import pyspark.sql.functions as sf

resultDF = (df
            .select(
                sf.col("id"),
                sf.lower(sf.col("company_name")).alias("company_name"),
                (sf.col("score") / 100).alias("score")
            )
           )

In [ ]:
resultDF.show()

In [ ]:
# Можно добавлять колонки
(
    resultDF
    .withColumn("company_name_upper", sf.upper(sf.col("company_name")))
    .show()
)

In [ ]:
# Можно перезаписывать существующие колонки
(
    resultDF
    .withColumn("company_name", sf.upper(sf.col("company_name")))
    .show()
)

In [ ]:
# После работы обязательно отключаем спарк и отдаем ресурсы!
spark.stop()